In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import math
import warnings
warnings.filterwarnings("ignore")

# Veri seti içerisinde bulunacak sütunlar

- datetime (tarih, saat, dakika)
- oda_tipi (Salon, Mutfak, Oturma Odası, Yatak Odası)
- sensor_tipi (sicaklik, nem, hava_kalitesi (belki daha sonra))
- sensor_degeri (full float)

In [3]:
n_days = 2
interval_minutes = 5

In [4]:
sensors = ["Temperature Sensor", "Humidity Sensor"]
locations = ["Living Room", "Kitchen", "Bedroom"]

start_time = datetime(2025, 8, 14, 9, 0)
end_time = start_time + timedelta(days=n_days)

timestamp = pd.date_range(start=start_time, end=end_time, freq=f"{interval_minutes}T") # freq parametresindeki T ifadesi minute olduğunu ifade ediyor

In [5]:
def generate_value(hour, base_min, base_max):
    # Sıcaklık yavaşça artıp yavaşça azalan bir şey olduğundan sıcaklık değerlerini üretmek için sinüs dalgası kullanılabilir
    # Nem değeri ise genellikle sıcaklık ile ters orantılıdır ancak zaten gerçek bir iot verisi üretmiyoruz o yüzden önemli değil
    sin_wave = math.sin(math.pi * (hour - 9) / 12) # sin(pi/2) değeri maksimum sıcaklığı verecek değerdir

    # Ortalama değerleri ve farkların ortalamasını hesaplıyoruz
    mid_point = (base_max + base_min) / 2
    amplitude = (base_max - base_min) / 2

    # Sinüs dalgasını kullanarak bir değer üretiyoruz
    value = mid_point + sin_wave * amplitude

    # Değeri biraz rastgeleleştiriyoruz
    value *= random.uniform(0.95, 1.05)

    # Son olarak elde edilen değer minimum ve maksimum aralığında mı bakıyoruz
    return round(max(base_min, min(base_max, value)), 2)

In [6]:
data = []
for i, time in enumerate(timestamp):
    for sensor in sensors:
        for location in locations: 
            temperature = generate_value(time.hour, 15.0, 27.0)
            humidity = generate_value(time.hour, 20, 80)
    
            if temperature > 32:
                status_temperature = "High Temperature"
            elif temperature < 22:
                status_temperature = "High Temperature"
            else:
                status_temperature = "Normal Temperature"

            if humidity > 60:
                status_humidity = "High Humidity"
            elif humidity < 30:
                status_humidity = "Low Humidity"
            else:
                status_humidity = "Normal Humidity"

            tarih = time.strftime('%Y-%m-%d')
            saat = time.strftime('%H:%M')
            
            data.append(
                {
                    "Date" : tarih,
                    "Hour" : saat,
                    "Sensor" : sensor,
                    "Room" : location,
                    "Sensor Value" : temperature if sensor == "Temperature Sensor" else humidity,
                    "Sensor Unit" : "°C" if sensor == "Temperature Sensor" else "%",
                    "Room Status" : status_temperature if sensor == "Temperature Sensor" else status_humidity
                }
            )
df = pd.DataFrame(data)

In [7]:
df.head(10)

,Date,Hour,Sensor,Room,Sensor Value,Sensor Unit,Room Status
0,2025-08-14,09:00,Temperature Sensor,Living Room,21.49,°C,High Temperature
1,2025-08-14,09:00,Temperature Sensor,Kitchen,21.84,°C,High Temperature
2,2025-08-14,09:00,Temperature Sensor,Bedroom,21.15,°C,High Temperature
3,2025-08-14,09:00,Humidity Sensor,Living Room,51.92,%,Normal Humidity
4,2025-08-14,09:00,Humidity Sensor,Kitchen,49.96,%,Normal Humidity
5,2025-08-14,09:00,Humidity Sensor,Bedroom,50.40,%,Normal Humidity
6,2025-08-14,09:05,Temperature Sensor,Living Room,20.69,°C,High Temperature
7,2025-08-14,09:05,Temperature Sensor,Kitchen,21.28,°C,High Temperature
8,2025-08-14,09:05,Temperature Sensor,Bedroom,21.49,°C,High Temperature
9,2025-08-14,09:05,Humidity Sensor,Living Room,50.23,%,Normal Humidity


In [8]:
df.to_csv("../Data/dummy_iot_dataset_last_en.csv", index = False)

In [9]:
txt_file = "../Data/iot_readable_last_en.txt"
csv_path = "../Data/dummy_iot_dataset_last_en.csv"

data = pd.read_csv(csv_path)

In [12]:
# Sadeleştirilmiş hâli (Similarity Searchte sade düzen daha iyi sonuç veriyor gereksiz kelimelerin olmaması modelin kolay anlamasına yardımcı oluyor)
with open(txt_file, "w", encoding="utf-8") as file:
    for index, row in data.iterrows():
        if row['Sensor Unit'] == '%':
            content = (
                f"Date: {row['Date']}, Hour: {row['Hour']}, Room: {row['Room']}, Sensor: {row['Sensor']}, Value: {row['Sensor Unit']}{row['Sensor Value']}, Status: {row['Room Status']}"
            )
        elif row['Sensor Unit'] == '°C':
            content = (
                f"Date: {row['Date']}, Hour: {row['Hour']}, Room: {row['Room']}, Sensor: {row['Sensor']}, Value: {row['Sensor Value']}{row['Sensor Unit']}, Status: {row['Room Status']}"
            )
        file.write(content+"\n")
print(f"{txt_file} dosyası başarılı bir şekilde oluşturuldu.")

../Data/iot_readable_last_en.txt dosyası başarılı bir şekilde oluşturuldu.
